# This notebook is to test a single batch run in ADAM

In [ ]:
from adam import Batches
from adam import BatchPropagation
from adam import BatchPropagations
from adam import PropagationParams
from adam import OpmParams
from adam import ConfigManager
from adam import Projects
from adam import RestRequests
from adam import AuthenticatingRestProxy
import time
import os

This sets up authenticated access to the server. It needs to be done before pretty much everything you want to do with ADAM.

In [ ]:
# Reads your config from a file in current directory. For instructions on setting this up, see config_demo notebook.
config = ConfigManager().get_config(environment='dev')
print(config)
auth_rest = AuthenticatingRestProxy(RestRequests(config['url']), config['token'])

## Example Inputs

In [ ]:
# Keplerian elements and sigma from John's initial Monte Carlo notebook
keplerian_elements = {
    'semi_major_axis_km': 2.2321102129031378,
    'eccentricity': 0.5836725969822691,
    'inclination_deg': 0.889253120260506,
    'ra_of_asc_node_deg': 38.46053156164339,
    'arg_of_pericenter_deg': 34.36938785036925,
    'mean_anomaly_deg': 136.4013846143255,
    'gm': 1.327124400419394E11
}

keplerian_sigma = {
    'semi_major_axis_km': 8e-5,
    'eccentricity': 1e-4,
    'inclination_deg': 1e-1,
    'ra_of_asc_node_deg': 8e-2,
    'arg_of_pericenter_deg': 8e-2,
    'mean_anomaly': 3e-2
}

### Set Parameters

Commented parameters are optional. Uncomment to use.

In [ ]:
propagation_params = PropagationParams({
    'start_time': '2028-11-01T00:00:00Z',   # propagation start time in ISO format
    'end_time': '2030-11-01T00:00:00.0000Z',     # propagation end time in ISO format

    'project_uuid': config['workspace'],

#     'step_size': 60 * 60,                   # default step size (seconds) is 86400
#     'propagator_uuid': '00000000-0000-0000-0000-000000000001',  # default force model is "all major bodies"
    'description': 'Monte Carlo',       # description of run

    'propagationType': 'MONTE_CARLO',
    'monteCarloDraws': 100000,
    'keplerianSigma': keplerian_sigma
})

opm_params = OpmParams({
    'epoch': '2028-11-01T00:00:00Z',
    'keplerian_elements': keplerian_elements,
            
#     'mass': 500.5,              # object mass
#     'solar_rad_area': 25.2,     # object solar radiation area (m^2)
#     'solar_rad_coeff': 1.2,     # object solar radiation coefficient
#     'drag_area': 33.3,          # object drag area (m^2)
#     'drag_coeff': 2.5,          # object drag coefficient
    
#     'covariance': covariance,   # object covariance
#     'perturbation': 3,          # sigma perturbation on state vector
#     'hypercube': 'FACES',       # hypercube propagation type
    
#     'originator': 'Robot',      # originator of run
#     'object_name': 'TestObj',   # object name
#     'object_id': 'test1234',    # object ID
})

### Submit and Run Propagation

In [ ]:
# Use the new runnable framework to run hypercube
batch = BatchPropagation(propagation_params, opm_params)
print("Submitting OPM:")
print(batch.get_opm_params().generate_opm())

In [ ]:
# Submit and wait until batch run is ready
batches_api = BatchPropagations(auth_rest)
print(batches_api._build_batch_propagation_creation_data(propagation_params, opm_params, 'de77f430-7a4f-407c-93c1-a99cc4cd2909'))
batch_uuid = batches_api.insert(batch, 'de77f430-7a4f-407c-93c1-a99cc4cd2909')

### Get Status and Parts Count

In [ ]:
# Get final status and parts count
print(batch)
parts_count = batch.get_summary().get_parts_count()
print(batch.get_state_summary())
print("Final state: %s, part count %s\n" % (batch.get_calc_state(), parts_count))

### Get Ephemeris of Specified Part

In [ ]:
# Get ephemeris of specified part
parts = batch.get_results().get_parts()
for p in parts:
  print("part {}:\n{}\n".format(p.get_part_index(), p.get_ephemeris()))

### Get ending state vector

In [ ]:
# Get the end state vector (uncomment to use)
end_state_vector = batch.get_results().get_end_state_vector()
print("State vector at the end of propagation:")
print(end_state_vector)